The code is written in Python and uses the DUNE API to upload data.
The uplouding data is consist of 2 main steps:
1.  **Data Preparation**: In this step, the data is prepared for uploading to DUNE
-   Create the table 
-   The data is converted to the required format.
-   The data is validated to ensure it meets the required standards.
2.  **Data Upload**: In this step, the prepared data is uploaded to DUNE
-   The data is sent to DUNE using the DUNE API.
-   The response from DUNE is handled to ensure the data was uploaded successfully.


In [1]:
import pandas as pd
import numpy as np
import requests
from requests import Request
import json
import csv
import dotenv
import os
from dotenv import load_dotenv, find_dotenv
from flipside import Flipside
import logging
# from degate.spot import Spot as Client
# from degate.lib.utils import config_logging
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
load_dotenv()
dotenv.load_dotenv('../../.env')
api_key = os.environ["DUNE_API_KEY"]
# headers = {"X-Dune-API-Key": api_key}
flipside_key = os.environ["FLIPSIDE_API_KEY"]

DATA PRAEPARATION

In [ ]:
# Create the table to work with
schema = [
    {"name": "pair", "type": "varchar"},
    {"name": "date", "type": "timestamp", "nullable": False},
    {"name": "open", "type": "double", "nullable": False},
    {"name": "high", "type": "double", "nullable": False}, 
    {"name": "low", "type": "double", "nullable": False}, 
    {"name": "close", "type": "double", "nullable": False}, 
    {"name": "volume", "type": "double", "nullable": False}, 
    {"name": "close_time", "type": "timestamp", "nullable": False},
    {"name": "quote_volume", "type": "double", "nullable": False},
    {"name": "trades", "type": "double", "nullable": False}
    ]
url = "https://api.dune.com/api/v1/table/create"

payload = {
    "namespace": you_user_name,
    "table_name": "degate_trades",
    "description": "daily trading data, sourced from https://app.degate.com/analytics?s=&utm_source=&uuid=a78ef6c3-f6b3-4e7a-89bd-2cf321bf90de",
    "schema": schema, 
    "is_private": False
}

headers = {
    "X-DUNE-API-KEY": api_key,
    "Content-Type": "application/json"
}


response = requests.request("POST", url, json=payload, headers=headers)

In [ ]:
df = pd.read_csv('degate_trades.csv')
# df0.drop('Unnamed: 0', axis=1)
print(df.columns.tolist())

['pair', 'date', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'trades']


In [56]:
# convert timestamp into ISO 8601 timestamp format
base = pd.DataFrame(df)
base['date']= pd.to_datetime(base['date'], unit='ms')
base['close_time'] = pd.to_datetime(base['close_time'], unit='ms')
base['date']
# base['close_time']
base1 = base.to_csv('degate_trades_time_adjusted.csv')

In [ ]:
# check columns 
df = pd.read_csv('degate_trades_time_adjusted.csv')
# print(df.columns.tolist())

# Drop "Unnamed: 0" column if it exists
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

# Ensure correct column order
expected_columns = ['pair', 'date', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'trades']
df = df[expected_columns]

# Save the cleaned CSV
df.to_csv("degate_trades_final.csv", index=False)

print("Fixed CSV saved as 'degate_trades_final.csv'. Try uploading it again.")

In [62]:
df = pd.read_csv('degate_trades_final.csv')
print(df.columns.tolist())

['pair', 'date', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'trades']


DATA UPLOAD

In [ ]:
# insert data into created table 
url = "https://api.dune.com/api/v1/table/kemasan/degate_trades/insert"

headers = {
    "X-DUNE-API-KEY": api_key,
    "Content-Type": "text/csv"
}

with open("degate_trades_final.csv", "rb") as data:
  response = requests.request("POST", url, data=data, headers=headers)
  print(response.text)

{"name":"dune.kemasan.degate_trades","rows_written":55972,"bytes_written":5305801}


In [ ]:

# url = "https://api.dune.com/api/v1/table/create"

# payload = {
#     "namespace": "kemasan",
#     "table_name": "degate_trades",
#     "description": "daily trading data, source: https://app.degate.com/analytics?s=&utm_source=&uuid=a78ef6c3-f6b3-4e7a-89bd-2cf321bf90de ",
#     "schema": [{"name": "date", "type": "timestamp"}, {"name": "dgs10", "type": "double", "nullable": True}],
#     "is_private": False
# }

# headers = {
#     "X-DUNE-API-KEY": api_key,
#     "Content-Type": "application/json"
# }


# response = requests.request("POST", url, json=payload, headers=headers)

In [ ]:
# test = pd.read_csv('test1.csv')
# test1 = pd.DataFrame(test)
# test1['date'] = pd.to_datetime(test1['date'], unit='s')
# test1 = test1.to_csv('test2.csv')



In [ ]:
# # check CSV file before insert step
# df = pd.read_csv('test2.csv')
# # df0.drop('Unnamed: 0', axis=1)
# print(df.columns.tolist())
# # Drop "Unnamed: 0" column if it exists
# if "Unnamed: 0" in df.columns:
#     df = df.drop(columns=["Unnamed: 0"])

# # Rename columns to match expected names
# df = df.rename(columns={"data": "date", "dsg10": "dgs10"})

# # Ensure correct column order
# expected_columns = ["date", "dgs10"]
# df = df[expected_columns]

# # Save the cleaned CSV
# df.to_csv("cleaned_file.csv", index=False)

# print("Fixed CSV saved as 'cleaned_file.csv'. Try uploading it again.")